phase -1 import data and define

In [1]:
import pandas as pd
import numpy as np

dataset_path = "vmCloud_data.csv"
Cloud_data = pd.read_csv(dataset_path)

In [2]:
print(Cloud_data.head())

                                  vm_id            timestamp  cpu_usage  \
0  c5215826-6237-4a33-9312-72c1df909881  2023-01-25 09:10:54  54.881350   
1  29690bc6-1f34-403b-b509-a1ecb1834fb8  2023-01-26 04:46:34  71.518937   
2  2e55abc3-5bad-46cb-b445-a577f5e9bf2a  2023-01-13 23:39:47        NaN   
3  e672e32f-c134-4fbc-992b-34eb63bef6bf  2023-02-09 11:45:49  54.488318   
4  f38b8b50-6926-4533-be4f-89ad11624071  2023-06-14 08:27:26  42.365480   

   memory_usage  network_traffic  power_consumption  \
0     78.950861       164.775973         287.808986   
1     29.901883              NaN         362.273569   
2     92.709195       203.674847         231.467903   
3     88.100960              NaN         195.639954   
4           NaN              NaN         359.451537   

   num_executed_instructions  execution_time  energy_efficiency task_type  \
0                     7527.0       69.345575           0.553589   network   
1                     5348.0       41.396040           0.349856 

In [3]:

print(Cloud_data.columns)

Index(['vm_id', 'timestamp', 'cpu_usage', 'memory_usage', 'network_traffic',
       'power_consumption', 'num_executed_instructions', 'execution_time',
       'energy_efficiency', 'task_type', 'task_priority', 'task_status'],
      dtype='object')


phase - 2 * preprocessing the data and normalizing the values and cleaning the data


In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the dataset
df = pd.read_csv("vmCloud_data.csv")

df['timestamp'] = pd.to_datetime(df['timestamp'])

# Handle missing values
numerical_cols = ['cpu_usage', 'memory_usage', 'network_traffic', 'power_consumption', 
                  'num_executed_instructions', 'execution_time', 'energy_efficiency']
imputer = SimpleImputer(strategy='mean')
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])

# For categorical columns, you might want to fill missing values with a placeholder or the most frequent value
categorical_cols = ['task_type', 'task_priority', 'task_status']
df[categorical_cols] = df[categorical_cols].fillna('missing')

# Normalize or Standardize the numerical data
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

print(df.columns)

Index(['vm_id', 'timestamp', 'cpu_usage', 'memory_usage', 'network_traffic',
       'power_consumption', 'num_executed_instructions', 'execution_time',
       'energy_efficiency', 'task_type', 'task_priority', 'task_status'],
      dtype='object')


In [24]:
import pandas as pd

# Assuming 'df' is already defined and includes a 'timestamp' and 'cpu_usage' column

# Convert 'timestamp' to datetime and set as index
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp'])  # Drop rows where 'timestamp' is NaT
df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)  # Sort the DataFrame by the index

# Apply rolling averages
df['cpu_usage_rolling_mean_3h'] = df['cpu_usage'].rolling(window='3h').mean()
df['cpu_usage_rolling_mean_6h'] = df['cpu_usage'].rolling(window='6h').mean()

# Ensure the index is unique before applying shift operations
if df.index.duplicated().any():
    df = df[~df.index.duplicated(keep='first')]  # Drop duplicates, keeping the first occurrence

# Apply shift operations without 'freq' to avoid reindexing issues
df['cpu_usage_lag_1h'] = df['cpu_usage'].shift(periods=1)  # Shift by 1 period instead of using 'freq'
df['cpu_usage_lag_1d'] = df['cpu_usage'].shift(periods=24)  # Assuming hourly data, shift by 24 periods for 1 day

# Extract time components
df['hour_of_day'] = df.index.hour
df['day_of_week'] = df.index.dayofweek

# Drop rows with NaN values created by rolling and lag features
df.dropna(inplace=True)

# Prepare a DataFrame for Prophet
prophet_df = df.reset_index()[['timestamp', 'cpu_usage']].rename(columns={'timestamp': 'ds', 'cpu_usage': 'y'})

# The 'df' DataFrame now includes both original and engineered features and can be used directly for models like Random Forest

In [25]:
from prophet import Prophet

# Initialize the Prophet model
model = Prophet()

# Fit the model with your DataFrame
model.fit(prophet_df)

# Create a future DataFrame for the next 365 days
future = model.make_future_dataframe(periods=365)

# Predict future values
forecast = model.predict(future)

# Plot the forecast including the trend and seasonality components
fig1 = model.plot(forecast)
fig2 = model.plot_components(forecast)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
00:11:52 - cmdstanpy - INFO - Chain [1] start processing
00:14:24 - cmdstanpy - INFO - Chain [1] done processing
